In [1]:
import os
from pathlib import Path
import numpy as np
import json
from loguru import logger
import platform
import re 
import nibabel as nib
import statistics
import sys

import mri_data
import monai_training

from reload_recursive import reload_recursive

In [2]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_3Tpioneer_bids, DataSet, Scan  # noqa: E402, F401
from mri_data import utils
from mri_data import file_manager as fm
from monai_training import preprocess  # noqa: E402
from monai_training.preprocess import DataSetProcesser

In [3]:
logger.remove()
logger.add(sys.stderr, level="INFO")

1

In [4]:
#! Set these variables
work_dir_name = "choroid_resegment1"

In [5]:
hostname = platform.node()
if hostname == "rhinocampus":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")

msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = training_work_dirs / work_dir_name
ensemble_out_dir = work_dir / "ensemble_output"

Scan the ensemble_out folder produced during the monai training to collect the predicted labels into a DataSet struct. Create another DataSet object with the same subjects and sessions but rooted in the main data location, then collect the manual labels 

In [6]:
ensemble_out_dataset = scan_3Tpioneer_bids(ensemble_out_dir, label="flair.t1_ensemble.nii.gz")
dataset = DataSet.dataset_like(ensemble_out_dataset)
dataset.dataroot = dataroot
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels("choroid_t1_flair", suffix_list=["CH", "ED", "DT"])
dataset = dataset_proc.dataset
dataset.sort()
ensemble_out_dataset.sort()

2024-12-11 15:59:38.111 | INFO     | monai_training.preprocess:prepare_labels:167 - Prepare Labels
  0%|          | 0/7 [00:00<?, ?it/s]2024-12-11 15:59:38.127 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label choroid_t1_flair-ED.nii.gz for Scan(subid=2097, sesid=20171223)
2024-12-11 15:59:38.133 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label choroid_t1_flair-CH.nii.gz for Scan(subid=1010, sesid=20180208)
2024-12-11 15:59:38.139 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label choroid_t1_flair-CH.nii.gz for Scan(subid=1163, sesid=20180907)
2024-12-11 15:59:38.147 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label choroid_t1_flair-CH.nii.gz for Scan(subid=1548, sesid=20210628)
2024-12-11 15:59:38.153 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label choroid_t1_flair-CH.nii.gz for Scan(subid=1033, sesid=20171117)
2024-12-11 15:59:39.040 | INFO     | monai_training.preprocess:p

Calculate Dice Scores

In [7]:
dice_scores = []
print("Dice Scores")
for scan, inference in zip(dataset, ensemble_out_dataset):
    man_label = fm.find_label(scan, "choroid_t1_flair", ["CH", "ED", "DT"])
    seg1 = nib.load(man_label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.dice_score(seg1, seg2)
    dice_scores.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(dice_scores) / len(dice_scores)))
print("Std: {:0.2f}".format(statistics.stdev(dice_scores)))

print(dice_scores)

Dice Scores
1010: 0.68
1033: 0.75
1152: 0.72
1163: 0.63
1548: 0.78
2097: 0.67
2164: 0.78
----------
Mean: 0.72
Std: 0.06
[0.6844474079013169, 0.7526676222328397, 0.7243532560214094, 0.6282231598687295, 0.7828544512482336, 0.6735943588791984, 0.7819075712881023]


In [21]:
iou = []
print("IoU")
for scan, inference in zip(dataset, ensemble_out_dataset):
    seg1 = nib.load(scan.root / "choroid_t1_flair.nii.gz").get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.iou(seg1, seg2, seg2_val=1)
    iou.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(iou) / len(iou)))
print("Std: {:0.2f}".format(statistics.stdev(iou)))

print(iou)

IoU
1010: 0.55
1033: 0.64
1152: 0.57
1163: 0.60
1548: 0.63
2097: 0.51
2164: 0.64
----------
Mean: 0.59
Std: 0.05
[0.546950354609929, 0.6434086359736917, 0.5678321678321678, 0.5996022538945973, 0.628143392188336, 0.5078343592613318, 0.6419115272844689]


Calculate Hausdorff Distance for Choroid

In [8]:
haus = []
print("Hausdorff Dist")
for scan, inference in zip(dataset, ensemble_out_dataset):
    man_label = fm.find_label(scan, "choroid_t1_flair", ["CH", "ED", "DT"])
    seg1 = nib.load(man_label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.hausdorff_dist(seg1, seg2, seg2_val=1)
    haus.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(haus) / len(haus)))
print("Std: {:0.2f}".format(statistics.stdev(haus)))

print(haus)

Hausdorff Dist
1010: 2.24
1033: 1.41
1152: 1.41
1163: 2.45
1548: 1.41
2097: 6.40
2164: 1.41
----------
Mean: 2.39
Std: 1.82
[2.23606797749979, 1.4142135623730951, 1.4142135623730951, 2.449489742783178, 1.4142135623730951, 6.4031242374328485, 1.4142135623730951]


Save a new version of the predicted label that has a labelling index of 2 so that it appears as a different color in itksnap

In [ ]:
inference: Scan = ensemble_out_dataset[0]
label_path = inference.label_path
new_label_path = label_path.with_name(f"{fm.nifti_name(label_path)}_val2.nii.gz")
utils.set_label_value(label_path, new_label_path, 2)

PosixPath('/home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble_val2.nii.gz')

In [10]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    t1 = scan.root / "t1.nii.gz"
    manual = scan.label_path
    prediction = inference.label_path
    print("itksnap", "-g", flair, "-o", t1, "-s", prediction, "-s", manual)

itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1033/ses-20171117/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1033/ses-20171117/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment1/ensemble_output/sub-ms1033/ses-20171117/flair.t1_ensemble.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1152/ses-20170529/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1152/ses-20170529/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment1/ensemble_output/sub-ms1152/ses-20170529/flair.t1_ensemble.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1163/ses-20180907/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1163/ses-20180907/t1.nii.gz -s /home/s

Produce commands to open images and labels in freeview

In [1]:
image_paths = ["one", "twp", "three"]
step1 = " -g ".join(image_paths)
step1.split(" ")

['one', '-g', 'twp', '-g', 'three']

In [ ]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    t1 = scan.root / "t1.nii.gz"
    manual = scan.label_path
    prediction = inference.label_path
    print("freeview", flair, t1, f"{prediction}:colormap=heat", f"{manual}:colormap=heat")

freeview /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/pineal-CH.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1029/ses-20170816/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/pineal-SRS.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1188/ses-20200720/flair.t1_ensemble.nii.gz:colormap=heat